In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import plotly.graph_objects as go

from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network

In [3]:
data = pd.read_csv("covid_19_continent.csv").rename(columns={
        "Unnamed: 0": "Date",
        "index": "Date"
    }).set_index("Date")
data.index = pd.to_datetime(data.index, format="%Y-%m-%d")

data.head()

,Africa,Americas,Asia,Boat,Europe,Oceania
Date,,,,,,
2020-01-22,0,1,556,0,0,0
2020-01-23,0,1,654,0,0,0
2020-01-24,0,2,937,0,2,0
2020-01-25,0,2,1428,0,3,0
2020-01-26,0,6,2105,0,3,4


In [8]:
px.line(data)

In [7]:
px.line(data.diff())